# Multivariate Regressions and The Naive Regression-Weighted Causal Effect

We consider the following Data Generating Process:
$$
x = \text{Uniform}(\{1,2,3,4\}) \\
T = \exp(x) + u, \quad u \sim N(0,\sigma^2) \\ 
Y = \sin(T) + x + v, \quad v \sim N(0,1) \\ 
$$

Notice that we have $T|x \sim N(\exp(x),\sigma^2)$ for each $x$. We can then calculate the conditional Yitzhaki's weights. Starting with the numerator, we have:
$$
E[T - E[T|x] \mid T > t, x] \cdot p(X > t \mid x) = [\exp(x) + \sigma \frac{\phi(t)}{1 - \Phi(t)} - \exp(x)] \cdot (1 - \Phi(t)) = \sigma \phi(t) \\
$$

Similarly, as the variance is fixed across $x$, we have that $E[\text{var}(T|x)] = \sigma^2$. Therefore, the weights are:
$$
w(t,x) = \frac{\sigma \phi(t)}{\sigma^2} = \frac{\phi(t)}{\sigma},
$$
which is the density of the normal distribution. Using the fact that $\frac{\partial \sin(t)}{\partial t} = \cos(t)$, we can simply estimate the NRWCE as:
$$
E[\cos(T)] \approx \frac{1}{n} \sum_{i=1}^n \cos(T_i).
$$

In what follows we generate Data and comapre the regressions coefficient $\beta$ in the following regression
$$
Y = \alpha + \beta T + \gamma x + \epsilon
$$
to the $NRwCE$ estimand. In the analysis we fix $\sigma=1$

#### Generating Data

In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols


In [2]:
# First block
df1 = pd.DataFrame({'x': np.repeat(np.arange(2, 6),1000000,axis=0)})
df1['T'] = np.exp(df1['x']) + np.random.normal(size=len(df1))
df1['sinT'] = np.sin(df1['T'])
df1['y'] = df1['sinT'] + df1['x'] + np.random.normal(size=len(df1))



Without knowin the underlying DGP one may consider estimating the following linear model 
$$
Y = \alpha + \beta T + \gamma x + \epsilon
$$ 

In [3]:
result1 = ols('y ~ T + x', df1).fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                 6.860e+05
Date:                Thu, 25 May 2023   Prob (F-statistic):               0.00
Time:                        21:05:39   Log-Likelihood:            -6.2420e+06
No. Observations:             4000000   AIC:                         1.248e+07
Df Residuals:                 3999997   BIC:                         1.248e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8376      0.003    537.237      0.0

As we know the DGP, we can estimate the true native regression-weighted causal effect here.

In [4]:
df1['dT'] = np.cos(df1['T'])
print("The estimated NRwCE is: " ,df1['dT'].mean())


The estimated NRwCE is:  -0.04811411819821981


So, as we can see, it's different and has a different sign from our regression coefficient.

Next, we can demonstrate the impact of bias and attenuation on the resulting regression coefficient. Please note that the numbers may be close but not exact, as our analysis was conducted at the population level, whereas here we are working with a sample.



In [5]:
ols_tx = ols('T ~ x', df1).fit()

df1['DeltaX'] = np.exp(df1['x']) - ols_tx.params['x']*df1['x'] - ols_tx.params['Intercept']
df1['linear_res_tx'] = df1['T'] - ols_tx.params['x']*df1['x'] - ols_tx.params['Intercept']
df1['res_tx'] = df1['T'] - np.exp(df1['x'])
df1['residualizedT'] = df1['T'] - ols_tx.params['x']*df1['x'] - ols_tx.params['Intercept']

cov_bias = df1[['y', 'DeltaX']].cov().iloc[0,1]
cov_causal = df1[['y', 'res_tx']].cov().iloc[0,1]
sd = df1['residualizedT'].std()
print('The beta coefficient: ',(cov_bias + cov_causal) / (sd**2))
print('The bias part: ',(cov_bias ) / (sd**2))
print('The causal part: ',(cov_causal) / (sd**2))
print('The attenuation: ',cov_causal - (cov_causal) / (sd**2))
print('The NRwCE: ',cov_causal )




The beta coefficient:  0.003907648070431813
The bias part:  0.0040178015467263566
The causal part:  -0.00011015347629454367
The attenuation:  -0.04723546437065927
The NRwCE:  -0.04734561784695381


so we can see that the causal effect was attenuated to almost zero, and most of what drives $\beta$ is is the bias term. 

Now, assume that 
$$
T = x + u, u \sim N(0,1) \\
Y = sin(T)+ sin(x) + v, v \sim N(0,1) 
$$
and we repeat the same analysis


In [6]:
# First block
df1 = pd.DataFrame({'x': np.repeat(np.arange(2, 6),1000000,axis=0)})
df1['T'] = df1['x'] + np.random.normal(size=len(df1))
df1['sinT'] = np.sin(df1['T'])
df1['y'] = df1['sinT'] + np.sin(df1['x']) + np.random.normal(size=len(df1))

result1 = ols('y ~ T + x', df1).fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                 2.214e+06
Date:                Thu, 25 May 2023   Prob (F-statistic):               0.00
Time:                        21:05:44   Log-Likelihood:            -6.1961e+06
No. Observations:             4000000   AIC:                         1.239e+07
Df Residuals:                 3999997   BIC:                         1.239e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.3897      0.002   1811.268      0.0

In [7]:
df1['dT'] = np.cos(df1['T'])
print("The estimated NRwCE is: " ,df1['dT'].mean())


The estimated NRwCE is:  -0.2691692260468451


In [8]:
ols_tx = ols('T ~ x', df1).fit()

df1['DeltaX'] = df1['x'] - ols_tx.params['x']*df1['x'] - ols_tx.params['Intercept']
df1['linear_res_tx'] = df1['T'] - ols_tx.params['x']*df1['x'] - ols_tx.params['Intercept']
df1['res_tx'] = df1['T'] - df1['x']
df1['residualizedT'] = df1['T'] - ols_tx.params['x']*df1['x'] - ols_tx.params['Intercept']

cov_bias = df1[['y', 'DeltaX']].cov().iloc[0,1]
cov_causal = df1[['y', 'res_tx']].cov().iloc[0,1]
sd = df1['residualizedT'].std()
print('The beta coefficient: ',(cov_bias + cov_causal) / (sd**2))
print('The bias part: ',(cov_bias ) / (sd**2))
print('The causal part: ',(cov_causal) / (sd**2))
print('The attenuation: ',cov_causal - (cov_causal) / (sd**2))
print('The NRwCE: ',cov_causal )




The beta coefficient:  -0.2689132794693612
The bias part:  0.0001238800399784524
The causal part:  -0.2690371595093397
The attenuation:  -9.649816335671746e-05
The NRwCE:  -0.2691336576726964


We can observe that the regression coefficient on variable T now provides us with the corrected causal effect, even though the effect of the control variable is not linear in the outcome equation. 




